!pip install ultralytics

!pip install deep-sort-realtime

!pip install helper

!pip install torch torchvision torchaudio

In [35]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [1]:
import torch
torch.cuda.is_available()

True

In [6]:
import datetime
from ultralytics import YOLO
import cv2
import pyvirtualcam
import math
import numpy as np
from KalmanFilter import KalmanFilter
import sys

sys.path.insert(0, '../pyNDI/ndi')
#pyNDI Import
import finder
import receiver
import lib

find = finder.create_ndi_finder()
NDIsources = find.get_sources()
recieveSource = NDIsources[0]
reciever = receiver.create_receiver(recieveSource)

# define some constants

GREEN = (0, 255, 0)
w_fhd = int(1920/2)
h_fhd = int(1080/2)
w_hd=int(1280/2)
h_hd=int(720/2)

KF = KalmanFilter(0.1, 1, 5, 1, 20, 20)
# Video from BGR to RGB
fmt = pyvirtualcam.PixelFormat.BGR
# load the pre-trained YOLOv8n model
model = YOLO("yolov8n-pose.pt")
model.to('cuda')

In [7]:
CONFIDENCE_THRESHOLD = 0.4
f_total = 0
f_segm = 0
debugging = False
testing = False
movement_threshold = 0.4

In [8]:
def bounds(x, y, relative_w, relative_h, real_w, real_h):
    xmin = 0
    xmax = 0
    ymin = 0
    ymax = 0
    if x < relative_w:
        xmin = 0
        xmax = relative_w*2
    elif x > (real_w-relative_w):
        xmin = real_w-(relative_w*2)
        xmax = real_w
    else:
        xmin = int(x-relative_w)
        xmax = int(x+relative_w)
    
    if y < relative_h:
        ymin = 0
        ymax = relative_h*2
    elif y > (real_h-relative_h):
        ymin = real_h-(relative_h*2)
        ymax = real_h
    else:
        ymin = int(y-relative_h)
        ymax = int(y+relative_h)
    return xmin, xmax, ymin, ymax

In [9]:
centroidX = w_fhd
centroidY = h_fhd
#keyPoints = [0, 5, 6]
(x1, y1) = KF.update([w_fhd, h_fhd])
with pyvirtualcam.Camera(width=1920, height=1080, fps=60, fmt=fmt) as cam:
    while True:
        # Fotograma vacio
        frameProcessed = np.zeros((cam.height, cam.width, 3), np.uint8)
        
        # Inicio de tiempo para medir
        start = datetime.datetime.now()
    
        # Captura de video
        frame = reciever.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)

        # Se corre YOLO
        detections = model(frame, 
                           classes=0, 
                           conf=CONFIDENCE_THRESHOLD, 
                           verbose=False)[0].keypoints.data.tolist()
        
        temp_centroidX = 0
        temp_centroidY = 0
        total_length = 0
        
        for data in detections:
            if len(data)>0:
                for i, info in enumerate(data):
                    temp_centroidX = temp_centroidX + data[i][0]
                    temp_centroidY = temp_centroidY + data[i][1]
                    if debugging:
                        cv2.circle(frame, (int(data[i][0]), int(data[i][1])), radius=5, color=(255, 0, 0), thickness=-1)

                total_length = total_length + len(data)

        if total_length > 0:
            temp_centroidX = temp_centroidX/total_length
            temp_centroidY = temp_centroidY/total_length - 150
            temp_distanceCenter = np.sqrt((temp_centroidX-centroidX)**2+(temp_centroidY-centroidY)**2)


            if temp_distanceCenter > movement_threshold:
                centroidX = int(temp_centroidX)
                centroidY = int(temp_centroidY)


        if len(detections) > 0:   
            focus = np.asarray(KF.predict()[0]).reshape(-1)
            (x1, y1) = KF.update([centroidX, centroidY])

        if debugging:
            cv2.circle(frame, (centroidX, centroidY), radius=5, color=(0, 0, 255), thickness=-1)
            cv2.circle(frame, (int(focus[0]), int(focus[1])), radius=5, color=(0, 255, 0), thickness=-1)
            
        # Calculo del enfoque 
        #xmin_p, xmax_p, ymin_p, ymax_p = bounds(int(focus[0]), int(focus[1]), w_hd, h_hd, 1920, 1080)
        xmin_p, xmax_p, ymin_p, ymax_p = bounds(int(focus[0]), h_hd+200, w_hd, h_hd, 1920, 1080)
        frame = frame[ymin_p:ymax_p, xmin_p:xmax_p]

        # Aumentado del tamaño
        frame = cv2.resize(frame, (1920, 1080), interpolation=cv2.INTER_CUBIC)
        
        if debugging:           
            # end time to compute the fps
            end = datetime.datetime.now()
            # show the time it took to process 1 frame
            total = (end - start).total_seconds()
            #print(f"Time to process 1 frame: {total * 1000:.0f} milliseconds")

            # calculate the frame per second and draw it on the frame
            fps = f"FPS: {1 / total:.2f}"
            cv2.putText(frame, fps, (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

        cam.send(frame)
        cam.sleep_until_next_frame()

    video_cap.release()

KeyboardInterrupt: 